1:  Mucosal healing and inflammatory bowel disease: Therapeutic implications and new targets
Megan Lynn Otte, Raju Lama Tamang, Julia Papapanagiotou, Rizwan Ahmad, Punita Dhawan, Amar B Singh
World J Gastroenterol. 2023 Feb 21; 29(7): 1157–1172.  Published online 2023 Feb 21. doi: 10.3748/wjg.v29.i7.1157
PMCID: PMC10011951

2:  Gut-Liver Immune Traffic: Deciphering Immune-Pathogenesis to Underpin Translational Therapy
Amber G. Bozward, Vincenzo Ronca, Daniel Osei-Bordom, Ye Htun Oo
Front Immunol. 2021; 12: 711217.  Published online 2021 Aug 25. doi: 10.3389/fimmu.2021.711217
PMCID: PMC8425300

3:  Severity of Psychologic Stressors Reflects Course of Crohn’s Disease in Two Siblings
Jessica Budiselic, Mary Sadlek, Keon Simpson
Cureus. 2021 Jul; 13(7): e16533.  Published online 2021 Jul 21. doi: 10.7759/cureus.16533
PMCID: PMC8378317




In [7]:
from Bio import Entrez

Entrez.email = 'your_email@example.com'

fetch_handle = Entrez.efetch(db='pmc', id=35301512, rettype='json', retmode='text')

output = fetch_handle.read()
output

'server-errorServer returned response code 404 Service Unavailable. Url: http%3A%2F%2F130.14.22.90%2Futils%2Fportal%2Fportaltoc.fcgi%3Fncbi_sid%3D6F1CBA0EE09BC26E%255FB8EESID%26ncbi_phid%3D322C225246E54735000025D1955E884B%252E1%252E1%252E3%252E3%26myncbi_id%3D0%26port%3Dlive404Service Unavailable<?apply-render exception?><?ncbi_sid 6F1CBA0EE09BC26E_B8EESID?><?ncbi_phid 322C225246E54735000025D1955E884B.1.1.3.3?><?hostname 21?><?ExcErrorId 864141053?><?ExcTime 07/03/2023 21:43:33?><?UserAgent NCBI-Portal NCBIHttpConnector (CXX Toolkit)?><?ExcRequestUrl http://130.14.22.90/utils/portal/portaltoc.fcgi?ncbi_sid=6F1CBA0EE09BC26E%5FB8EESID&#x00026;ncbi_phid=322C225246E54735000025D1955E884B%2E1%2E1%2E3%2E3&#x00026;myncbi_id=0&#x00026;port=live?><exception/>'

In [2]:
event = {"search":"Vedolizumab"}
lambda_handler(event, "context")

[]

In [8]:
import requests
import random
import PyPDF2
import io


def lambda_handler(event, context):
    url = event['url']  # Assuming the URL is passed as an input in the event

    userAgents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/113.0",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/114.0",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Safari/605.1.15",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/114.0",
        "Mozilla/5.0 (Windows NT 10.0; rv:114.0) Gecko/20100101 Firefox/114.0",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.43",
        "Mozilla/5.0 (X11; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 OPR/99.0.0.0",
        "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/114.0",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/113.0"
    ]

    headers = {"User-Agent": random.choice(userAgents)}

    session = requests.Session()
    session.headers.update(headers)

    try:
        response = session.get(url, stream=True)
        response.raise_for_status()  # Raise an exception if the request was not successful

        pdf_content = response.content

        # Create a file-like object from the PDF content
        pdf_file = io.BytesIO(pdf_content)

        # Parse the PDF content using PyPDF2
        pdf = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf.pages:
            text = page.extract_text()
    except Exception as e:
        print(f"Error occurred while processing PDF: {str(e)}")
        text = "N/A"

    return {
        'statusCode': 200,
        'fulltext': text
    }
    
    
    



In [9]:
event = {"url": "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6010939/pdf/WJG-24-2457.pdf"}
lambda_handler(event, "context")

{'statusCode': 200,
 'fulltext': '                                      © 2018 Baishideng Publishing Group Inc . All rights reserved.Published by Baishideng Publishing Group Inc\n7901 Stoneridge Drive, Suite 501, Pleasanton, CA 94588, USA\nTelephone: +1-925-223-8242\nFax: +1-925-223-8243\nE-mail: bpgoffice@wjgnet.com\nHelp Desk: http://www.f6publishing.com/helpdesk\nhttp://www.wjgnet.com\nI S S N  1 0  0 7  -   9  3 2  7\n9    7 7 1 0  07   9 3 2 0 452  3'}

In [33]:
import requests
from Bio import Entrez
from bs4 import BeautifulSoup
import boto3

#client = boto3.client('lambda')

# Provide your email address to the Entrez API
Entrez.email = 'your_email@example.com'

def fetch_pmc_articles():
    fetch_handle = Entrez.efetch(db='pmc', id=9178865, rettype='json', retmode='text')
    return fetch_handle.read()


def lambda_handler(event, context):

    url = f"https://www.ncbi.nlm.nih.gov/pmc/?term={event['search']}"

    # Send a GET request to the URL
    response = requests.get(url)

    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the "dd" elements
    dd_elements = soup.find_all("dd")

    pmc_ids = []

    # Iterate over each "dd" element
    for dd in dd_elements:
        # Check if the text starts with "PMC"
        if dd.text.startswith("PMC"):
            # Remove the "<dd>" and "</dd>" tags and append the PMC ID
            pmc_id = dd.text.replace("<dd>", "").replace("</dd>", "").strip()
            pmc_ids.append(pmc_id)
            pmc_set = set(pmc_ids)

        xml_content = fetch_pmc_articles(pmc_set)
        print(xml_content)

In [34]:

event = {"search": "Vendolizumab"}
lambda_handler(event, "context")

In [1]:
import requests
from Bio import Entrez
from bs4 import BeautifulSoup
import boto3

#client = boto3.client('lambda')

# Provide your email address to the Entrez API
Entrez.email = 'your_email@example.com'

fetch_handle = Entrez.efetch(db='pmc', id=[9178865, 9322578, 9426668], rettype='xml', retmode='text')
fetch_handle.read()

b'<?xml version="1.0" ?>\n<!DOCTYPE pmc-articleset PUBLIC "-//NLM//DTD ARTICLE SET 2.0//EN" "https://dtd.nlm.nih.gov/ncbi/pmc/articleset/nlm-articleset-2.0.dtd">\n<pmc-articleset><article xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:ali="http://www.niso.org/schemas/ali/1.0/" article-type="research-article" dtd-version="1.3">\n  <?properties open_access?>\n  <processing-meta base-tagset="archiving" mathml-version="3.0" table-model="xhtml" tagset-family="jats">\n    <restricted-by>pmc</restricted-by>\n  </processing-meta>\n  <front>\n    <journal-meta>\n      <journal-id journal-id-type="nlm-ta">BMC Gastroenterol</journal-id>\n      <journal-id journal-id-type="iso-abbrev">BMC Gastroenterol</journal-id>\n      <journal-title-group>\n        <journal-title>BMC Gastroenterology</journal-title>\n      </journal-title-group>\n      <issn pub-type="epub">1471-230X</issn>\n      <publisher>\n        <publisher-name>BioMed Central</publisher-na

'1:  Comparative efficacy and safety of infliximab and vedolizumab therapy in patients with inflammatory bowel disease: a systematic review and meta-analysis\nLaurent Peyrin-Biroulet, Perttu Arkkila, Alessandro Armuzzi, Silvio Danese, Jordi Guardiola, Jørgen Jahnsen, Charles Lees, Edouard Louis, Milan Lukáš, Walter Reinisch, Xavier Roblin, Minyoung Jang, Han Geul Byun, Dong-Hyeon Kim, Sung Jeong Lee, Raja Atreya\nBMC Gastroenterol. 2022; 22: 291.  Published online 2022 Jun 8. doi:\xa010.1186/s12876-022-02347-1\nPMCID: PMC9178865\n\n2:  Real‐world data on switching from intravenous to subcutaneous vedolizumab treatment in patients with inflammatory bowel disease\nViktoria Bergqvist, Johanna Holmgren, Daniel Klintman, Jan Marsal\nAliment Pharmacol Ther. 2022 Jun; 55(11): 1389–1401.  Published online 2022 Apr 25. doi:\xa010.1111/apt.16927\nPMCID: PMC9322578\n\n3:  Pharmacokinetics, Safety and Efficacy of Intravenous Vedolizumab in Paediatric Patients with Ulcerative Colitis or Crohn’s Disease: Results from the Phase 2 HUBBLE Study\nJeffrey S Hyams, Dan Turner, Stanley A Cohen, Erzsébet Szakos, Kinga Kowalska-Duplaga, Frank Ruemmele, Nicholas M Croft, Bartosz Korczowski, Promise Lawrence, Siddharth Bhatia, Harisha Kadali, Chunlin Chen, Wan Sun, Maria Rosario, Senthil Kabilan, William Treem, Guillermo Rossiter, Richard A Lirio\nJ Crohns Colitis. 2022 Aug; 16(8): 1243–1254.  Published online 2022 Mar 17. doi:\xa010.1093/ecco-jcc/jjac036\nPMCID: PMC9426668\n\n'


In [76]:
import re
import requests
from Bio import Entrez
from bs4 import BeautifulSoup
import boto3
from datetime import datetime

# client = boto3.client('lambda')

def lambda_handler(event, context):
    url = f"https://www.ncbi.nlm.nih.gov/pmc/?term={event['search']}"

    # Send a GET request to the URL
    response = requests.get(url)

    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the "dd" elements
    dd_elements = soup.find_all("dd")

    pmc_ids = []

    # Iterate over each "dd" element
    for dd in dd_elements:
        # Check if the text starts with "PMC"
        if dd.text.startswith("PMC"):
            # Remove the "<dd>" and "</dd>" tags and append the PMC ID
            pmc_id = dd.text.replace("<dd>", "").replace("</dd>", "").strip()
            pmc_ids.append(pmc_id)
            pmc_set = set(pmc_ids)

    # Provide your email address to the Entrez API
    Entrez.email = 'your_email@example.com'

    fetch_handle = Entrez.efetch(db='pmc', id=pmc_set, rettype='json', retmode='text')

    output = fetch_handle.read()

    # Splitting the output into individual paper entries
    articles = output.split('\n\n')

    # Regular expression patterns for extracting information
    title_pattern = r'\d+: (.*?)[\n,.]'
    authors_pattern = r'\n(.*?)\n'
    date_pattern = r'Published online (\d{4} [A-Za-z]+ \d{1,2})\.'
    doi_pattern = r"doi:\xa0(\S+)"
    pmcid_pattern = r"PMCID: (\S+)"
    
    paper_list = []

    # Extracting the information from each paper
    for article in articles[:-1]:  # Exclude the last empty element
        
        #dictionary for this paper:
        paper = {}   
        
        title_match = re.findall(title_pattern, article)
        paper['title'] = title_match[0] if title_match else None
        
        authors_match = re.findall(authors_pattern, article)
        paper['authors'] = authors_match[0] if authors_match else None
        
        date_match = re.findall(date_pattern, article)
        paper['date'] = date_match[0] if date_match else None
        
        if paper['date']:
            date_obj = datetime.strptime(paper['date'], '%Y %b %d')
            paper['date'] = date_obj.strftime('%Y/%m/%d')
        else:
            paper['date'] = '0000/00/00'  # Assign a default value for sorting
        
        doi_match = re.findall(doi_pattern, article)
        paper['doi'] = doi_match[0] if doi_match else None
        
        pmcid_match = re.findall(pmcid_pattern, article)
        paper['pmcid'] = pmcid_match[0] if pmcid_match else None
        paper['pdflink'] = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{paper['pmcid']}/pdf"

        paper_list.append(paper)
        
    json_response = {
        'statusCode': 200,
        'body': {
            'papers': sorted(paper_list, key=lambda x: x['date'], reverse=True)
        }
    }
    return json_response


In [77]:
event = {"search": "Entyvio"}
lambda_handler(event, "context")

{'statusCode': 200,
 'body': {'papers': [{'title': ' Comparative efficacy and safety of infliximab and vedolizumab therapy in patients with inflammatory bowel disease: a systematic review and meta-analysis',
    'authors': 'Laurent Peyrin-Biroulet, Perttu Arkkila, Alessandro Armuzzi, Silvio Danese, Jordi Guardiola, Jørgen Jahnsen, Charles Lees, Edouard Louis, Milan Lukáš, Walter Reinisch, Xavier Roblin, Minyoung Jang, Han Geul Byun, Dong-Hyeon Kim, Sung Jeong Lee, Raja Atreya',
    'date': '2022/06/08',
    'doi': '10.1186/s12876-022-02347-1',
    'pmcid': 'PMC9178865',
    'pdflink': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9178865/pdf'},
   {'title': ' Real‐world data on switching from intravenous to subcutaneous vedolizumab treatment in patients with inflammatory bowel disease',
    'authors': 'Viktoria Bergqvist, Johanna Holmgren, Daniel Klintman, Jan Marsal',
    'date': '2022/04/25',
    'doi': '10.1111/apt.16927',
    'pmcid': 'PMC9322578',
    'pdflink': 'https://www.ncbi.